In [90]:
# For altering memory usage
from sage.interfaces.gap import get_gap_memory_pool_size, set_gap_memory_pool_size
n = get_gap_memory_pool_size()
set_gap_memory_pool_size(200*n)
n = get_gap_memory_pool_size()
n

15137757560000

In [158]:
# Groups 
# Cyclic groups definition C[x].
C = [CyclicPermutationGroup(1),CyclicPermutationGroup(1)]
for i in range(30):
     C.append(CyclicPermutationGroup(i+2))

# Althernating group definition A[x].
A = [AlternatingGroup(4), AlternatingGroup(4), AlternatingGroup(4), AlternatingGroup(4)]
for i in range(15):
     A.append(AlternatingGroup(i+4))

# List of indices of small groups by GAP library: [x,y].
small = []
for i in range(2,50):
    if i+1 not in [64,96,128]:
        small += gap.IdsOfAllSmallGroups(i+1)

In [92]:
# Function to return list of class sizes in default order.
def class_sizes(G):
    group_order = G.order()
    reps = G.conjugacy_classes_representatives()
    class_sizes = []
    for g in reps:
        class_sizes.append(group_order / G.centralizer(g).order())
    return class_sizes

In [93]:
# Function to find all subgroups. Returns list of subgroups.
def subgroups(G):
    subgroups = G.conjugacy_classes_subgroups()
    return subgroups

In [94]:
# Function to find induced trivial character of each subgroup. Returns a list of lists.
def get_induced(subgroups, G):
    ind = []
    for subgroup in subgroups:
        ind.append(subgroup.trivial_character().induct(G))
    return ind

In [155]:
# Forms the matrix that we use to solve the simultaneous equation later. Excludes the first and last subgroup (trivial subgroup and the group itself), and the first column.
def mat(induced):
    matrix = []
    checkmat = []
    for char in induced:
        matrix.append(list(char.values())[1:])
        checkmat.append(list(char.values())[0])
#    for line in matrix:
#        print(line)
    return matrix[1:-1], checkmat[1:-1]

In [163]:
# Test if there exists non-negative solutions. Prints the id of group that works and the ci, prints fail otherwise.
def test(G):
    a = class_sizes(G)
    b = subgroups(G)
    c = get_induced(b, G)
    d, iden = mat(c)
    e = Matrix(d).T
    temp = []
    for i in range(len(a)-1):
        temp.append(1)
    Y = vector(temp)
    p = MixedIntegerLinearProgram(solver='GLPK')
    w = p.new_variable(nonnegative=True)
    p.add_constraint(e*w == Y)
    p.solve()
    try:
        p.solve()
        out = []
        #Used to check identity
        check = []
        for i, v in (p.get_values(w).items()):
            out.append('w_%s = %s' % (i, v))
            check.append(v)
            # Prints subgroup and subgroup order.
            #if round(v,6) != 0:
            #    print(b[i+1],b[i+1].order())
        #Check if identity is larger than others.
        x = 0
        for i in range(len(iden)):
            x += int(iden[i])*float(check[i])
        # Prints the solution and SmallGroup Id
        if x > 1:
            print(out)
            print(gap.IdSmallGroup(G))
        else:
            print("Check this group:" + gap.IdSmallGroup(G))
    except:
        print('fail')

In [165]:
# Main function
# If group is in id form.
# https://www.gap-system.org/Manuals/pkg/SmallGrp-1.3/doc/chap1.html
# This works on non-windows system with gap_packages library installed.

groups = small
for groupid in groups:
    G = gap.SmallGroup(groupid)
    #test(PermutationGroup(gap_group = G.AsPermGroup()))
    try:
        test(PermutationGroup(gap_group = G.AsPermGroup()))
    except:
        pass

# If group is in SAGE form.
# https://doc.sagemath.org/html/en/thematic_tutorials/group_theory.html#groups-of-small-order-as-permutation-groups
"""
D1 = CyclicPermutationGroup(2)
D2 = CyclicPermutationGroup(2)
G = direct_product_permgroups([D1,D2])
groups = [direct_product_permgroups([D1,D1,D1,D1,D1,D1])]
for group in groups:
    try:
        test(group)
    except:
        pass"""

3.0
['w_0 = 0.5', 'w_1 = 0.5', 'w_2 = 0.5']
[ 4, 2 ]
4.0
['w_0 = 1.0', 'w_1 = 0.5']
[ 6, 1 ]


5.0
['w_0 = 0.5', 'w_1 = 0.0', 'w_2 = 0.5', 'w_3 = 0.0', 'w_4 = 0.0', 'w_5 = 0.5']
[ 8, 3 ]


7.0
['w_0 = 0.25', 'w_1 = 0.25', 'w_2 = 0.25', 'w_3 = 0.25', 'w_4 = 0.25', 'w_5 = 0.25', 'w_6 = 0.25', 'w_7 = 0.0', 'w_8 = 0.0', 'w_9 = 0.0', 'w_10 = 0.0', 'w_11 = 0.0', 'w_12 = 0.0', 'w_13 = 0.0']
[ 8, 5 ]
4.0
['w_0 = 0.3333333333333333', 'w_1 = 0.3333333333333333', 'w_2 = 0.3333333333333333', 'w_3 = 0.3333333333333333']


[ 9, 2 ]
6.0
['w_0 = 1.0', 'w_1 = 0.5']
[ 10, 1 ]


7.0
['w_0 = 0.5', 'w_1 = 1.0', 'w_2 = 0.0']
[ 12, 3 ]
7.0
['w_0 = 0.0', 'w_1 = 0.5', 'w_2 = 0.5', 'w_3 = 0.0', 'w_4 = 0.0', 'w_5 = 0.5', 'w_6 = 0.0', 'w_7 = 0.0']
[ 12, 4 ]


8.0
['w_0 = 1.0', 'w_1 = 0.5']
[ 14, 1 ]


RuntimeError: Gap produced error output
Error, Variable: '$sage810' must have a value

   executing __SAGE_LAST__:="__SAGE_LAST__";;SmallGroup(\$sage810);;